## MODULES

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from scipy.stats import cauchy
from IPython.display import display, clear_output

## Functions and test

In [11]:

# Funciones para generar trayectorias
def generate_CRW(steps, speed, initial_position, cauchy_coef):
    angles = cauchy.rvs(0, cauchy_coef, size=steps)
    x = np.cos(angles) * speed
    y = np.sin(angles) * speed
    z = np.random.normal(0, speed, size=steps)  # Z con distribución normal
    changes = np.vstack([x, y, z]).T
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory

def generate_LF(steps, speed, initial_position, cauchy_coef, alpha):
    distances = np.random.pareto(alpha, size=steps) + speed
    angles = cauchy.rvs(0, cauchy_coef, size=steps)
    x = np.cos(angles) * distances
    y = np.sin(angles) * distances
    z = np.random.normal(0, speed, size=steps)  # Z con distribución normal
    changes = np.vstack([x, y, z]).T
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory

def generate_BM(steps, speed, initial_position):
    changes = np.random.normal(0, speed, size=(steps, 3))
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory


## Metrics

In [12]:
# Functions to calculate metrics
def calculate_route_length(trajectory):
    distances = np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))
    total_length = np.sum(distances)
    return total_length

def calculate_mean_square_displacement(trajectory):
    displacement = trajectory - trajectory[0]
    msd = np.mean(np.sum(displacement**2, axis=1))
    return msd

## Function to plot

In [13]:
# Function to plot trajectory and metric
def plot_trajectory_and_metric(trajectory, metric_values, metric_name):
    plt.figure(figsize=(14, 6))

    # 3D Trajectory
    ax1 = plt.subplot(1, 2, 1, projection='3d')
    ax1.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2])
    ax1.set_title('3D Trajectory')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')

    # Metric in line plot
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(metric_values, label=metric_name, color='purple')
    ax2.set_title(f'{metric_name} over time')
    ax2.set_xlabel('Step')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()

